In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import log_loss

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [3]:
train_df.head(2)

,id,Mean_Integrated,SD,EK,Skewness,Mean_DMSNR_Curve,SD_DMSNR_Curve,EK_DMSNR_Curve,Skewness_DMSNR_Curve,Class
0,0,133.171875,59.716081,0.043133,-0.703383,54.917224,70.084438,0.749798,-0.649512,0
1,1,87.093750,36.257973,0.435469,2.266057,3.417224,21.865069,7.039330,52.686251,0


In [4]:
test_df.head(2)

,id,Mean_Integrated,SD,EK,Skewness,Mean_DMSNR_Curve,SD_DMSNR_Curve,EK_DMSNR_Curve,Skewness_DMSNR_Curve
0,117564,140.046875,54.507800,0.058862,-0.567263,2.337793,14.868335,9.591760,117.988781
1,117565,107.828125,51.578965,0.284368,-0.338430,1.574415,12.501437,11.694968,182.704822


In [5]:
X_train_val = train_df.drop(["id", "Class"], axis=1).to_numpy()
X_test = test_df.drop(["id"], axis=1).to_numpy()

In [6]:
y_train_val = train_df.Class.to_numpy()

In [7]:
sc = StandardScaler()
X_train_val = sc.fit_transform(X_train_val)
X_test = sc.transform(X_test)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.10, random_state=42)

In [9]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape)

(105807, 8) (105807,)
(11757, 8) (11757,)
(78377, 8)


In [10]:
XGBClassifier?

Init signature:
XGBClassifier(
    *,
    objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'binary:logistic',
    use_label_encoder: Optional[bool] = None,
    **kwargs: Any,
) -> None
Docstring:     
Implementation of the scikit-learn API for XGBoost classification.


Parameters
----------

    n_estimators : int
        Number of boosting rounds.

    max_depth :  Optional[int]
        Maximum tree depth for base learners.
    max_leaves :
        Maximum number of leaves; 0 indicates no limit.
    max_bin :
        If using histogram-based algorithm, maximum number of bins per feature
    grow_policy :
        Tree growing policy. 0: favor splitting at nodes closest to the node, i.e. grow
        depth-wise. 1: favor splitting at nodes with highest loss change.
    learning_rate : Optional[float]
        Boosting learning rate (xgb's "eta")
    verbosity : Optional[int]
        The degree of verbosity. Valid values are 

In [19]:
model = XGBClassifier(learning_rate=0.06, n_estimators=700, max_depth=4, n_jobs=-1, objective='binary:logistic', eval_metric="logloss", early_stopping_rounds=150)
model.fit(x=X_train, y= y_train, eval_set=(X_val, y_val))

ValueError: too many values to unpack (expected 2)

In [12]:
predicated_val_ans = model.predict(X_val)
val_accuracy = accuracy_score(y_val, predicated_val_ans)
print("Validation accuracy is %.2f%%" % (val_accuracy*100))

Validation accuracy is 99.08%


In [13]:
predicated_test_ans = model.predict(X_test)
print(predicated_test_ans)

[0 0 0 ... 0 0 1]


In [14]:
log_loss(y_val, predicated_val_ans)

0.31727588995242845

In [ ]:
ans_df = pd.DataFrame({"id": test_df.id, "Class": predicated_test_ans})
ans_df.to_csv("outputs/ans16.csv", index=False)